In [22]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

from imblearn.over_sampling import SMOTENC
from collections import Counter

from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics

import umap

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import hypertools as hyp

import sys
sys.path.append(os.path.abspath('..'))

from util import evaluate_model_performance, evaluate_model_fairness

In [2]:
data_path = Path(os.getcwd()).parent.parent / "data" / "dataset_diabetes"
df = pd.read_csv(data_path / "data_analyzed.csv")

In [3]:
df["age"] = df["age_all"]

columns_to_remove = ['encounter_id', 'patient_nbr', 'readmitted', 'readmit_binary', 'diabetes_type', \
    'had_emergency', 'had_inpatient_days', 'had_outpatient_days', 'race_all', 'age_all']

df_for_experimenting = df.drop(columns=columns_to_remove)

In [4]:
data_path_write = Path(os.getcwd()).parent / "fawos" / "FAWOS" / "datasets" / "diabetes"
df_for_experimenting.to_csv(data_path_write / "raw_dataset.csv")

In [5]:
target_variable = "readmit_30_days"
Y = df_for_experimenting.loc[:, target_variable]
X = df_for_experimenting.drop(columns=[target_variable])

In [6]:
X.head() # sanity check

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,age_numeric
0,Caucasian,Female,[0-10),Unknown,Other,Other,Referral,1,Unknown,Other,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),Unknown,Emergency,Discharged to Home,Emergency,3,Unknown,Missing,...,No,Up,No,No,No,No,No,Ch,Yes,10
2,AfricanAmerican,Female,[20-30),Unknown,Emergency,Discharged to Home,Emergency,2,Unknown,Missing,...,No,No,No,No,No,No,No,No,Yes,20
3,Caucasian,Male,[30-40),Unknown,Emergency,Discharged to Home,Emergency,2,Unknown,Missing,...,No,Up,No,No,No,No,No,Ch,Yes,30
4,Caucasian,Male,[40-50),Unknown,Emergency,Discharged to Home,Emergency,1,Unknown,Missing,...,No,Steady,No,No,No,No,No,Ch,Yes,40


## Oversampling - SMOTENC

In [7]:
categorical_features = ['race', 'gender', 'weight', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', \
'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', \
'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', \
'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone','tolazamide', 'examide', 'citoglipton', 'insulin','glyburide-metformin', \
'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed']

col_idx_mapping = zip(df_for_experimenting.columns, range(len(df_for_experimenting.columns)))
col_idx_filtered = list(filter(lambda x: x[0] in categorical_features, col_idx_mapping))
idx_filtered = list(map(lambda x: x[1], col_idx_filtered))

In [8]:
X_one_hot = pd.get_dummies(X)

In [9]:
random_seed = 445
np.random.seed(random_seed)

X_train, X_test, X_train_one_hot, X_test_one_hot, Y_train, Y_test = train_test_split(
    X,
    X_one_hot,
    Y,
    test_size=0.20,
    stratify=Y,
    random_state=random_seed
)

In [10]:
sm = SMOTENC(random_state=42, categorical_features=list(idx_filtered))
X_train_res_one_hot, Y_train_res = sm.fit_resample(X_train_one_hot, Y_train)
print(f'Resampled dataset samples per class {Counter(Y_train_res)}')

Resampled dataset samples per class Counter({False: 72324, True: 72324})


### Logistic regression

In [11]:
lr_res = LogisticRegression(solver='newton-cg')
lr_res.fit(X_train_res_one_hot, Y_train_res)

# Predicting on the test data
lr_pred_test_res = lr_res.predict(X_test_one_hot)
evaluate_model_performance(Y_test, lr_pred_test_res)
evaluate_model_fairness(Y_test, lr_pred_test_res, X_test['race'])

The accuracy score for the testing data: 0.8868962806465878
The precision score for the testing data: 0.26865671641791045
The recall score for the testing data: 0.007926023778071334
The F1 score for the testing data: 0.015397775876817793
The F2 score for the testing data: 0.00983499071139766
The specificity score for the testing data: 0.9972901227740294
The balanced accuracy score for the testing data: 0.5026080732760504
The G mean score for the testing data: 0.08890750939455361
[[18033    49]
 [ 2253    18]]
The Demographic parity difference score for the testing data: 0.0038792820040765338
The Equalized odds difference score for the testing data: 0.008782201405152224
The Equal opportunity difference score for the testing data: 0.007926023778071334


c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
metrics.RocCurveDisplay.from_predictions(Y_test, lr_pred_test_res)

### Decision tree

In [12]:
tree_auto_balanced_res = DecisionTreeClassifier()
tree_auto_balanced_res.fit(X_train_res_one_hot, Y_train_res)

# Predicting on the test data
tree_pred_test_res = tree_auto_balanced_res.predict(X_test_one_hot)
evaluate_model_performance(Y_test, tree_pred_test_res)
evaluate_model_fairness(Y_test, tree_pred_test_res, X_test['race'])

The accuracy score for the testing data: 0.7511914705448828
The precision score for the testing data: 0.13680104031209364
The recall score for the testing data: 0.23161602818141788
The F1 score for the testing data: 0.1720078482668411
The F2 score for the testing data: 0.20341867120426949
The specificity score for the testing data: 0.8164472956531357
The balanced accuracy score for the testing data: 0.5240316619172768
The G mean score for the testing data: 0.43485891946542743
[[14763  3319]
 [ 1745   526]]
The Demographic parity difference score for the testing data: 0.035539148042830454
The Equalized odds difference score for the testing data: 0.09878333049064758
The Equal opportunity difference score for the testing data: 0.23161602818141788


c:\Users\Tanya\Desktop\TANYA\FMI\Masters\thesis\src\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
feat_importances = pd.DataFrame(tree_auto_balanced_res.feature_importances_[:10], index=X_train_res_one_hot.columns[:10], columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
(feat_importances * 100).head()

# feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
# feat_importances.plot(kind='bar', figsize=(20,5))

,Importance
age_numeric,6.906785
num_medications,4.821714
num_procedures,4.719751
num_lab_procedures,4.678536
time_in_hospital,3.552492


### Perceptron

In [ ]:
perceptron_res = Perceptron()
perceptron_res.fit(X_train_res_one_hot, Y_train_res)

# Predicting on the test data
perceptron_pred_test_res = perceptron_res.predict(X_test_one_hot)
evaluate_model_performance(Y_test, perceptron_pred_test_res)
evaluate_model_fairness(Y_test, perceptron_pred_test_res, X_test['race'])

### SVM (linear kernel)

In [ ]:
svm_res = LinearSVC()
svm_res.fit(X_train_res_one_hot, Y_train_res)

# Predicting on the test data
svm_pred_test_res = svm_res.predict(X_test_one_hot)
evaluate_model_performance(Y_test, svm_pred_test_res)
evaluate_model_fairness(Y_test, svm_pred_test_res, X_test['race'])

### Gaussian Naive Bayes

In [ ]:
nbc = GaussianNB()
nbc.fit(X_train_res_one_hot, Y_train_res)

# Predicting on the test data
nbc_pred_test_res = nbc.predict(X_test_one_hot)
evaluate_model_performance(Y_test, nbc_pred_test_res)
evaluate_model_fairness(Y_test, nbc_pred_test_res, X_test['race'])

### t-SNE visualization

In [ ]:
tsne_embedding = TSNE(n_components=2, learning_rate=50, init='random', perplexity=50).fit_transform(pd.get_dummies(X))

In [ ]:
tsne_embedding_oversampled_smotenc = TSNE(n_components=2, learning_rate=50, init='random', perplexity=50).fit_transform(X_res)

In [ ]:
colors = ['#F3DC1B', '#F37A3F', '#EC1DF3', '#27C4F1', '#00F37A', "red", "blue", "purple", 'green', 'yellow', 'pink', 'cyan', 'magenta', 'orange', 'grey', 'black']

sns.set()
_, axes = plt.subplots(1, 2, figsize=(15, 8))

sns.scatterplot(
    ax=axes[0],
    x=tsne_embedding[:, 0], y=tsne_embedding[:, 1],
    hue=list(Y),
    data=tsne_embedding,
    legend="full",
    alpha=0.2,
    palette=colors
)

sns.scatterplot(
    ax=axes[1],
    x=tsne_embedding_oversampled_smotenc[:, 0], y=tsne_embedding_oversampled_smotenc[:, 1],
    hue=list(Y_res),
    data=tsne_embedding_oversampled_smotenc,
    legend="full",
    alpha=0.2,
    palette=colors
)

### UMAP

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding_umap = reducer.fit_transform(pd.get_dummies(X))

In [ ]:
embedding_umap_resampled = reducer.fit_transform(X_res)

In [ ]:
sns.set()
_, axes = plt.subplots(1, 2, figsize=(15, 8))

sns.scatterplot(
    ax=axes[0],
    x=embedding_umap[:, 0], y=embedding_umap[:, 1],
    hue=list(Y),
    data=embedding_umap,
    legend="full",
    alpha=0.2
)

sns.scatterplot(
    ax=axes[1],
    x=embedding_umap_resampled[:, 0], y=embedding_umap_resampled[:, 1],
    hue=list(Y_res),
    data=embedding_umap_resampled,
    legend="full",
    alpha=0.2
)

### PCA - 2d

In [ ]:
pca = PCA(n_components=2, whiten=True) 
X_pca = pca.fit_transform(pd.get_dummies(X))
X_resampled_pca = pca.fit_transform(X_res)

In [ ]:
sns.set()
_, axes = plt.subplots(1, 2, figsize=(15, 8))

sns.scatterplot(
    ax=axes[0],
    x=X_pca[:, 0], y=X_pca[:, 1],
    hue=list(Y),
    data=X_pca,
    legend="full",
    alpha=0.2
)

sns.scatterplot(
    ax=axes[1],
    x=X_resampled_pca[:, 0], y=X_resampled_pca[:, 1],
    hue=list(Y_res),
    data=X_resampled_pca,
    legend="full",
    alpha=0.2
)

### PCA - 3d

In [ ]:
pca = PCA(n_components=3, whiten=True) 
X_pca_3d = pca.fit_transform(pd.get_dummies(X))
X_resampled_pca_3d = pca.fit_transform(X_res)

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(X_pca_3d[:, 0], X_pca_3d[:, 1], X_pca_3d[:, 2])
plt.title("simple 3D scatter plot")
 
# show plot
plt.show()

### Hypertools (should be cited)

In [ ]:
X_reduced = hyp.reduce(x=X, reduce='IncrementalPCA', ndims=10)

In [ ]:
X_tsne_double_reduced = hyp.reduce(x=X_reduced, reduce='TSNE', ndims=3)

In [ ]:
hyp.plot(X_tsne_double_reduced, '.', hue=Y, save_path='.')

In [ ]:
hyp.plot(X_res, '.', hue=Y_res, reduce='FastICA')